# Simulating the Role Passport on cadCAD

## Part 1: Pre-requisites

In [1]:
from dataclasses import dataclass
from functools import reduce
from typing import NamedTuple

## Part 2: Module Logic

## Part 3: cadCAD Simulation

In [1]:
"""
Tier 1: Proof of Relevance
Tier 2: Proof of Membership
Tier 3: Proof of Impact
"""

AttestationUUID = str
Days = int

class AttestationProbabilityArgs(NamedTuple):
    attest_probability: float
    invoke_probability: float


@dataclass
class Attestation():
    """
    TODO: add docstring
    """
    attestation_uuid: AttestationUUID
    result: bool  # Between 0.0 and 1.0
    issuance: Days
    expiry: Days


@dataclass
class PassportDemoAgent():
    """
    TODO: add docstring
    """
    attestations: list[Attestation]

    def valid_attestations(self, reference_time) -> list[Attestation]:
        """
        TODO: add docstring
        """
        return [a
                for a in self.attestations
                if a.expiry < self.reference_time
                and a.issuance >= self.reference_time]


@dataclass
class RoleCertificator():
    """
    TODO: add docstring
    """
    attestation_uuid: AttestationUUID
    issuance_validity: Days

    conditional_qualifiers: dict[str, float]
    auto_qualifers: set[str]
    disqualifiers: set[str]

    qualifier_aggregator: callable[[dict], float]
    conditional_threshold: float

    def generate_attestation(self, result: bool, time: Days) -> Attestation:
        """
        TODO: add docstring
        """
        return Attestation(self.uuid,
                           result,
                           time,
                           time + self.issuance_validity)

    def agent_certification_result(self,
                                   agent: PassportDemoAgent,
                                   time: Days) -> bool:
        """
        TODO: add docstring
        """
        agent_sucessful_attestations = set(a.source
                                           for a in agent.valid_attestations(time)
                                           if a.result == True)

        if len(agent_sucessful_attestations & self.disqualifiers) > 0:
            return False
        elif len(agent_sucessful_attestations & self.auto_qualifers) > 0:
            return True
        else:
            conditional_attestations = set(self.conditional_qualifiers.keys())
            agent_relevant_attestations = agent_sucessful_attestations & conditional_attestations

            agent_conditional_values = {a: self.conditional_qualifiers[a]
                                        for a
                                        in agent_relevant_attestations}
            agent_conditional_value = self.qualifier_aggregator(
                agent_conditional_values)
            if agent_conditional_value >= self.conditional_threshold:
                return True
            else:
                return False

    def attestate_agent(self, agent: PassportDemoAgent, time: Days):
        return self.generate_attestation(self.agent_certification_result(agent, time), time)


compound_product = lambda lst: reduce(lambda x, y: (1 + x) * (1 + y) - 1, lst)

Role_A = RoleCertificator(attestation_uuid='role_A_attestation',
                         issuance_validity=90,
                         conditional_qualifiers={'att_1': 1, 'att_2': 2, 'att_3': 3},
                         auto_qualifers={},
                         disqualifiers={'att_ban'},
                         qualifier_aggregator=lambda atts: sum(atts.values()),
                         conditional_threshold=2)

Role_B = RoleCertificator(attestation_uuid='role_B_attestation',
                         issuance_validity=30,
                         conditional_qualifiers={'att_3': 0.3, 'att_4': 0.2, 'att_5': 0.1},
                         auto_qualifers={'role_A_attestaion'},
                         disqualifiers={'att_ban'},
                         qualifier_aggregator=lambda atts: compound_product(atts.values()),
                         conditional_threshold=0.5)

AttestationProviders = {
    'att_1': AttestationProbabilityArgs(0.8, 0.2),
    'att_2': AttestationProbabilityArgs(0.5, 0.8),
    'att_3': AttestationProbabilityArgs(0.1, 0.5),
    'att_4': AttestationProbabilityArgs(0.5, 0.5),
    'att_5': AttestationProbabilityArgs(0.5, 0.5),
    'att_ban': AttestationProbabilityArgs(0.1, 0.1)
}


@dataclass
class RolePassportDemoModelState():
    """
    TODO: add docstring
    """
    agents: list[PassportDemoAgent]


@dataclass
class RolePassportDemoModelParams():
    """
    TODO: add docstring
    """
    days_per_timestep: Days
    attestation_providers: list[AttestationProbabilityArgs]
    role_certifactors: list[RoleCertificator]


model_psubs = [
    {
        'label': 'Time tracking'
    },
    {
        'label': "Agents trigger Attestation Providers"
    },
    {
        'label': 'Agents trigger Role Certificators'
    }
]


NameError: name 'dataclass' is not defined

## Part 4: Results